### HMM: CpG Islands
Using python for finding HMM (hidden Markov model) given observed data. Unfair coin example used (Which can be transformed to CpG Islands problem).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame

Define original model:

In [2]:
coin_vs = ["fair", "unfair"]
coin_ps = [[0.93, 0.07],[0.3, 0.7]]

# None means unknown:
flip_vs = ["head", "tails"]
flip_ps = [[0.5, 0.5],[0.9, 0.1]]


def sample(N, transition_matrix, transition_values,
           emission_matrix, emission_values, states):
    N -= 1
    if N<=0:
        return(DataFrame(states))
    
    previus_transition = states[-1]["transition"]
    previus_emission = states[-1]["emission"]

    p_t = transition_matrix[transition_values.index(previus_transition)]
    # print(p_t)
    coin = np.random.choice(transition_values, p=p_t)
    
    p_e = emission_matrix[emission_values.index(previus_emission)]
    flip = np.random.choice(emission_values, p=p_e)
    
    return(sample(N, transition_matrix, transition_values,
                  emission_matrix, emission_values,
                  states+[{"transition": coin, "emission": flip}]))

Generate observed data:

In [3]:
N = 900
data = sample(N, coin_ps, coin_vs, flip_ps, flip_vs,
              [{"transition":"fair", "emission":"head"}])
data[:10]

,transition,emission
0,fair,head
1,fair,head
2,fair,head
3,unfair,tails
4,unfair,head
5,unfair,head
6,unfair,tails
7,unfair,head
8,fair,head
9,fair,head


In [4]:
obs = np.array(list(map(lambda x: 0 if x == "head" else 1, data["emission"])))

Using `bayespy`:

In [ ]:
from bayespy.nodes import Dirichlet                                   
from bayespy.nodes import CategoricalMarkovChain                      
from bayespy.nodes import Categorical, Mixture   

Define init model:

In [11]:
a0 = [0.6, 0.4] 
# a0 = Dirichlet(1e-3*np.ones(2))

A = [[0.5, 0.5], 
     [0.5, 0.5]] 
# A = Dirichlet(1e-3*np.ones((2, 2)))
Z = CategoricalMarkovChain(a0, A, states=N)
# Z = CategoricalMarkovChain(a0, A, states=N,plates=(N,))

P = [[0.5, 0.5],
     [0.5, 0.5]]

# b0 = Dirichlet(1e-3*np.ones(2))
# B = Dirichlet(1e-3*np.ones((2, 2)))

Y = Mixture(Z, Categorical, P)
# Y = Mixture(Z, Dirichlet)
# Y = Mixture(Z, Dirichlet, 1e-3*np.ones((2,2)), plates=(N,))

Set observed variables:

In [13]:
Y.observe(obs)

Using variational Bayesian inference:

In [14]:
from bayespy.inference import VB

In [15]:
Q = VB(Y, Z)

In [16]:
Q.update(repeat=1000)

Iteration 1: loglike=-6.238325e+02 (0.405 seconds)
Iteration 2: loglike=-6.238325e+02 (0.384 seconds)
Converged at iteration 2.


Results:

In [17]:
print(Y.parents[0].get_moments()[0][-1])
print(Z.parents[1].get_moments())

[0.5 0.5]
[array([[-0.69314718, -0.69314718],
       [-0.69314718, -0.69314718]])]


In [19]:
'''
import pymc

xt1 = pymc.Categorical('xt1',[0.5,0.5]) 

# Priors on unknown parameters:
alpha = pymc.Normal('alpha',mu=0,tau=.01)

xt1 = pymc.Categorical('xt1',[0.5,0.5])                              

@pymc.deterministic
def theta(xt1=xt1,):
    return [0.5, 0.5 if]
'''

"\nimport pymc\n\nxt1 = pymc.Categorical('xt1',[0.5,0.5]) \n\n# Priors on unknown parameters:\nalpha = pymc.Normal('alpha',mu=0,tau=.01)\n\nxt1 = pymc.Categorical('xt1',[0.5,0.5])                              \n\n@pymc.deterministic\ndef theta(xt1=xt1,):\n    return [0.5, 0.5 if]\n"